[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/tutorial/notebooks/frederick_ipts/)

<img src='__docs/__all/notebook_rules.png' />

# Select IPTS

In [ ]:
from __code.frederick_ipts import FrederickIpts

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_file_metadata_display.ui')
from __code.file_metadata_display import Interface

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

In [ ]:
%gui qt

# Select Files 

In [ ]:
o_fred = FrederickIpts(working_dir = system.System.get_working_dir())
o_fred.select_files()

# Display images 

In [ ]:
o_gui = Interface(exp_dict=o_fred.exp_dict)
o_gui.show()